# Reorganize rain gauge data
This notebook is for performing reorganizing of the rain gauge data from being stored in csv files to being stored as a single dataset.

In [8]:
import numpy as np
import pandas as pd
import os
import datetime

import xarray as xr

In [2]:
f = open('../path_to_data.txt', "r")
path_to_dataset = os.path.join(f.readline().rstrip("\n"))
f.close() 

In [3]:
dataloc = os.path.join(path_to_dataset, 'real_rain_gauge_data', 'csv_formatado/')
metadata = pd.read_pickle(os.path.join(path_to_dataset, 'real_rain_gauge_data', 'general_rain_gauge_info.pckl'))

In [4]:
# Start and end dates of period to consider
st = '2020-12-01'
et = '2021-01-01'

In [5]:
# Create dataframe with all gauges
for ind in metadata.index:
    print(ind)
    df = pd.read_csv(os.path.join(dataloc,metadata.filename[ind]), delimiter = ',', header=0)
    df.DATA_UTC = pd.to_datetime(df.DATA_UTC, format='%Y-%m-%d')
    mask = (df.DATA_UTC >= st)&(df.DATA_UTC < et)
    nf = df.loc[mask]
    if(len(nf.DATA_UTC)>0):
        for date in nf.DATA_UTC:
            tmp = nf[nf.DATA_UTC==date].drop(columns='DATA_UTC').T
            tmp.columns = [ind]
            tmp_t = [date+pd.Timedelta(hours=int(i)) for i in list(tmp.index)]
            tmp.insert(0, 'timestamp', tmp_t)
            
            if(date==nf.DATA_UTC.iloc[0]):
                h = tmp.copy()
            else:
                h = h.append(tmp, ignore_index=True)

        if(ind==0):
            tot = h.copy()
        else:
            tot = pd.merge(tot, h, how='outer', on=['timestamp', 'timestamp']) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
tot

,timestamp,0,1,2,3,4,5,6,7,8,...,599,600,601,602,603,604,605,606,607,608
0,2020-12-01 00:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN
1,2020-12-01 01:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2,2020-12-01 02:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0
3,2020-12-01 03:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,2020-12-01 04:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2020-12-31 19:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,0.0,0.2,0.0,NaN,0.0,NaN,NaN,0.0
740,2020-12-31 20:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0
741,2020-12-31 21:00:00,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0
742,2020-12-31 22:00:00,0.2,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0


In [9]:
# Convert to xarray dataset
real = tot.copy()
real = real.set_index('timestamp', drop=True)
datasets = []
for key in real.columns:
    realsubs = real[[key]]
    rs = realsubs.to_xarray()
    rs = rs.rename(name_dict={'timestamp': 'time'})
    gauge_loc = list(rs.data_vars.keys())[0]
    rs = rs.rename_vars(name_dict={gauge_loc: 'gauge_precip'})
    rs = rs.assign_coords(dict(location=int(gauge_loc)))
    rs = rs.expand_dims('location', axis=1)
    rs = rs.set_coords(['time', 'location'])
    print(rs)
    datasets.append(rs)
real_comb = xr.concat(datasets, dim='location')

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 0
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.2 0.6
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 1
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 2
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 94
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 95
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.2 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 96
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * locat

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 170
Data variables:
    gauge_precip  (time, location) float64 1.0 3.6 2.6 2.2 ... 0.2 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 171
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 1.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 172
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * lo

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 251
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 252
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 253
Data variables:
    gauge_precip  (time, location) float64 nan nan nan 0.0 ... 0.0 2.2 17.2 2.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * l

    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 331
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 332
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 333
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 444
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 445
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 446
Data variables:
    gauge_precip  (time, location) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * lo

<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 583
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 584
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... nan nan nan nan
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 585
Data variables:
    gauge_precip  (time, location) float64 nan nan nan nan ... 0.4 0.6 0.0 0.2
<xarray.Dataset>
Dimensions:       (location: 1, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * lo

In [10]:
real_comb

<xarray.Dataset>
Dimensions:       (location: 588, time: 744)
Coordinates:
  * time          (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
  * location      (location) int64 0 1 2 3 4 5 6 ... 602 603 604 605 606 607 608
Data variables:
    gauge_precip  (time, location) float64 0.0 nan nan nan ... 0.0 nan nan 0.0

In [11]:
real_comb.to_netcdf(os.path.join(path_to_dataset,'real_rain_gauge_data','real_rain_gauge_data.nc'))